# Análise de Ações da S&P 500: Cotação + Notícias

**Objetivo**  
Dado um ticker de empresa listada na S&P 500, extrair:
1. Nome da empresa  
2. Cotação atual  
3. Últimas _n_ notícias (título e resumo)  

E depois fazer uma análise simples (comprimento dos títulos, frequência de palavras nos resumos).


## Introdução

Neste notebook vamos demonstrar como extrair, de forma totalmente automatizada, 
a cotação atual de qualquer ação da S&P 500 diretamente do Yahoo Finance, 
e em seguida fazer _web-scraping_ das últimas notícias relacionadas a essa empresa, 
também no Yahoo. Finalmente, faremos uma pequena análise exploratória de texto.


In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import re

%matplotlib inline

HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

## Metodologia

1. Montar URL base do Yahoo Finance:  
   `https://finance.yahoo.com/quote/{ticker}`  
2. Fazer `requests.get` e parsear com BeautifulSoup.  
3. Extrair:
   - **Nome da empresa** no `<h1>` do topo.
   - **Preço atual** no primeiro `<fin-streamer data-field="regularMarketPrice">`.
   - **Notícias**: nos primeiros `<li>` de classe `js-stream-content`, pegar `<h3>` (título) e `<p>` (resumo).
4. Consolidar em um `dict` + `DataFrame`.
5. Análise simples: histogramas e _word cloud_.


In [8]:
def fetch_page(ticker: str):
    url = f"https://finance.yahoo.com/quote/{ticker}/"
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    return soup

def clean_company_name(name: str) -> str:
    cleaned_name = re.sub(r"\s*\(.*?\)", "", name)
    return cleaned_name.strip()


def get_company_name(soup):
    h1_tag = soup.find("h1", class_="yf-xxbei9")
    if h1_tag:
        raw_name = h1_tag.text.strip()
        return clean_company_name(raw_name)
    return "Nome não encontrado"

# — Cell: extração de preço atual —
def get_current_price(soup):
    price_span = soup.find("span", {"data-testid": "qsp-price"})
    if price_span:
        return price_span.text.strip()
    return None

def get_latest_news(ticker: str, n: int = 5) -> pd.DataFrame:
    rss_url = (
        "https://feeds.finance.yahoo.com/rss/2.0/headline"
        f"?s={ticker}&region=US&lang=en-US"
    )
    r = requests.get(rss_url, headers=HEADERS)
    r.raise_for_status()
    xml = BeautifulSoup(r.text, "xml")
    items = xml.find_all("item")[:n]
    dados = []
    for it in items:
        dados.append({
            "titulo":  it.title.text.strip(),
            "resumo":  it.description.text.strip()
        })
    df = pd.DataFrame(dados, columns=["titulo","resumo"])
    return df


In [9]:
def analyze_stock(ticker: str, n_news: int = 5) -> dict:
    soup = fetch_page(ticker)
    return {
        "ticker":      ticker,
        "nome":        get_company_name(soup),
        "preco_atual": get_current_price(soup),
        "noticias":    get_latest_news(ticker, n=n_news)
    }

In [10]:
res = analyze_stock("AAPL", n_news=5)
print(f"{res['nome']} ({res['ticker']}) → Cotação: {res['preco_atual']}")
res["noticias"]


Apple Inc. (AAPL) → Cotação: 208.31


,titulo,resumo
0,Apple Partner Foxconn Taps India With Massive ...,Foxconn's Bold ?127.7 Billion India Gamble as ...
1,"Amer Sports, General Motors, Apple: Trending T...",Amer Sports (AS) stock pops after beating firs...
2,Alphabet Traders Look to Developer Event to Sh...,(Bloomberg) -- Alphabet Inc.’s investors are l...
3,Cloud AI Today - Azure Managed Redis Boosts Pe...,Microsoft has announced the general availabili...
4,"1 Unstoppable Stock That Could Join Nvidia, Ap...",Nine American companies are worth $1 trillion ...


## Extração para vários tickers

Você pode carregar uma lista de tickers da S&P 500 (ex: CSV oficial) ou usar um subset manual abaixo.

In [11]:
tickers = ["AAPL", "MSFT", "GOOGL"]
todos = [analyze_stock(t, n_news=3) for t in tickers]

df_prices = pd.DataFrame([
    {"ticker": r["ticker"], "nome": r["nome"], "preco_atual": r["preco_atual"]}
    for r in todos
])
df_prices

,ticker,nome,preco_atual
0,AAPL,Apple Inc.,208.35
1,MSFT,Microsoft Corporation,455.85
2,GOOGL,Alphabet Inc.,166.83


## Análise exploratória de Sentimento
Vamos ver a análise de sentimento da ação baseado nas notícias relacionados ao Ticker da ação.

In [12]:
import openai
from openai import AzureOpenAI

In [13]:
client = AzureOpenAI(
    api_key="13JQ7jW0IvRU38ZvFr4WK12I2guNFINq5Xv1dyLxsj6E9HksmNH2JQQJ99BDACfhMk5XJ3w3AAAAACOGLSYU",
    api_version="2024-12-01-preview",
    azure_endpoint="https://valer-m8yo6ng0-swedencentral.cognitiveservices.azure.com/openai/deployments/gpt-4o_MacielVidal_Chave1/chat/completions?api-version=2025-01-01-preview"
)

In [14]:
def sentimento_geral_noticias(df_noticias, cotacao, client, deployment_name="gpt-4o_MacielVidal_Chave1"):
    """
    Analisa o sentimento geral de todas as notícias e da cotação usando a API do ChatGPT.
    Retorna uma string com o sentimento geral e uma breve explicação.

    Parâmetros:
        df_noticias: DataFrame com colunas 'titulo' e 'resumo'
        cotacao: valor atual da cotação da ação (str ou float)
        client: instância do AzureOpenAI já autenticada
        deployment_name: nome do deployment do modelo (default: gpt-4o_MacielVidal_Chave1)
    """
    # Junta todas as notícias em um único texto
    textos = []
    for idx, row in df_noticias.iterrows():
        textos.append(f"Título: {row['titulo']}\nResumo: {row['resumo']}")
    texto_completo = "\n\n".join(textos)

    prompt = (
        "Considere as notícias e a cotação abaixo sobre uma empresa. "
        "Analise o sentimento geral (positivo, negativo ou neutro) e explique brevemente o porquê. "
        "Leve em conta tanto o conteúdo das notícias quanto o valor da cotação. "
        "Responda em português.\n\n"
        f"Cotação atual: {cotacao}\n\n"
        f"{texto_completo}"
    )

    response = client.chat.completions.create(
        model=deployment_name,
        messages=[{"role": "user", "content": prompt}],
        max_tokens=150,
        temperature=0
    )
    return response.choices[0].message.content.strip()

In [17]:
# Lista de tickers das empresas que você quer analisar
tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "META"]

resultados = []

for ticker in tickers:
    res = analyze_stock(ticker, n_news=5)
    sentimento = sentimento_geral_noticias(res["noticias"], res["preco_atual"], client)
    resultados.append({
        "ticker": ticker,
        "nome": res["nome"],
        "cotacao": res["preco_atual"],
        "sentimento": sentimento
    })

# Exibir os resultados
for r in resultados:
    print(f"\n--- {r['nome']} ({r['ticker']}) ---")
    print(f"Cotação: {r['cotacao']}")
    print(f"\n{r['sentimento']}")


--- Apple Inc. (AAPL) ---
Cotação: 208.39

**Sentimento geral: Negativo**

**Análise:**

1. **Notícia 1 (Foxconn e Índia):** A notícia destaca um investimento significativo da Foxconn na Índia, o que pode ser visto como positivo para a diversificação da cadeia de suprimentos da Apple. No entanto, o contexto de "Apple Retreats from China" (Apple recua da China) sugere desafios na relação com o mercado chinês, o que pode ser interpretado como um sinal de dificuldades estratégicas.

2. **Notícia 2 (Exportações de iPhone):** A menção de que as exportações de iPhones da China para os EUA atingiram o menor nível em 14 anos é um dado negativo, indicando possíveis

--- Microsoft Corporation (MSFT) ---
Cotação: 455.92

**Sentimento geral: Positivo**

**Análise:**

1. **Cotação atual**: A cotação de 455.92 é um valor elevado, indicando que a empresa está bem avaliada no mercado. Além disso, o aumento do preço-alvo para $540 pelo Citi sugere otimismo em relação ao desempenho futuro da Microsoft.

## Conclusão

- Extraímos nome, preço e notícias de qualquer ação da S&P 500 usando só requests+BS4.  
- Vimos como histogramas e contagem de palavras dão insight rápido sobre o texto.  
- Em próximos passos, poderíamos adicionar análise de sentimento, séries históricas de preço, etc.
